Código utilizado na resolução do exercicio 3. Esste notebook está um tanto desorganizado, mas acredito que dá pra ter uma ideia geral. Basicamente eu dividi os dados por tipo de prova e para cada tipo de prova apliquei um treinamento (Decision tree com gridsearch para tunagem do hiperparametros). Este notebook não é a versão final da qual fiz o envio para o codenation, mas é o mais perto que consegui recuperar na minha máquina (não estava no github. Alguns outros algoritimos alem do Decision Tree foram experimentados também.

In [1]:
%matplotlib inline
%pylab inline

# Importar bibliotecas
import numpy as np
import pandas as pd
import warnings
import json

from time import time
from sklearn.metrics import f1_score, r2_score
from hmmlearn import hmm
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, cross_val_score, train_test_split

warnings.filterwarnings(action='once')
    
# Ler os dados dos estudantes
student_data = pd.read_csv("train.csv")
print("Os dados dos estudantes foram lidos com êxito!")

Populating the interactive namespace from numpy and matplotlib
Os dados dos estudantes foram lidos com êxito!


In [2]:
gb_map = {'A': 1, 'B':2, 'C':3, 'D':4, 'E':5, '*':0, '.':0}

gabarito = {
    '706866896147fc1fe97fb4351bc789a04b3e1ac8':'CEECDBDADEBCCEBABECEACBDABDBDDABDDECADCCEACBA',
    '33afbd61183d36159e198672aea0957d3988f398':'DADCCECBDECADCBACDBCEEABDDAABDBCEADDBECEBAEBC',
    'eaf8decfa5e293b30347d68e8c96bd308d1747e2':'CBAACCEABDDCDBDDCEAECADBECEBACBDABDBEBCDADCEE', 
    '7768972b2a16ffd60c566951859cb2742cbbea50':'CCEEBCCEBAABDDECADCEECDBCBDBECBADDACEAABDBDAD',
    '533c7c38726cb52f1075e2699c3e7d6b4b95a2e5':'BCABBDDBDCDCCCACEDCBADEEEECAACEACEBCBDEBBBDDD',
    'c5d07ed2a1535f074eb558a440b069b5eefa6246':'BDDDBDEBBCEBCEACAACEEEEACEDCBADCCDCDCDBABBDBC',
    'ab155449f2d11d09b533b92c9547aaf26a264a34':'BDEBBCAACCBADCCACEDCEBCBDDDEEEEBCEADCDCDBABBD',
    'e0406d1d2dd9b6f07e2db00602f9dc663450ea51':'CAACDBCEBCBDDDEEEEBCABBDBDEBBEADCDCCCACEDCBAD',
    #'284fac346c24f267c62cf9070dd8bb1ee23e213c': 'CEECDBDADEBCCEBABECEACBDABDBDDABDDECADCCEACBA'
   }


map_f = lambda x: gb_map[x]

def prepare_gb(gb):
    return np.asarray(list(map(map_f, gb)))


gabaritos =[prepare_gb(list(gabarito['706866896147fc1fe97fb4351bc789a04b3e1ac8'])),
            prepare_gb(list(gabarito['33afbd61183d36159e198672aea0957d3988f398'])),
            prepare_gb(list(gabarito['eaf8decfa5e293b30347d68e8c96bd308d1747e2'])),
            prepare_gb(list(gabarito['7768972b2a16ffd60c566951859cb2742cbbea50'])),
            prepare_gb(list(gabarito['533c7c38726cb52f1075e2699c3e7d6b4b95a2e5'])),
            prepare_gb(list(gabarito['c5d07ed2a1535f074eb558a440b069b5eefa6246'])),
            prepare_gb(list(gabarito['ab155449f2d11d09b533b92c9547aaf26a264a34'])),
            prepare_gb(list(gabarito['e0406d1d2dd9b6f07e2db00602f9dc663450ea51'])),
            #prepare_gb(list(gabarito['284fac346c24f267c62cf9070dd8bb1ee23e213c']))
           ]

In [18]:
g1 = student_data[student_data['CO_PROVA_MT'] == '706866896147fc1fe97fb4351bc789a04b3e1ac8']

print(list(g1['TX_RESPOSTAS_MT'].values[0]))
print(list(g1['TX_GABARITO_MT'].values[0]))
print(g1['NU_NOTA_MT'].values[0])

for index in range(10):
    respostas = prepare_gb(list(g1['TX_RESPOSTAS_MT'].values[index]))
    gabarito = prepare_gb(list(g1['TX_GABARITO_MT'].values[index]))

    display(respostas-gabarito)

['C', 'A', 'C', 'D', 'D', 'D', 'B', 'A', 'D', 'B', 'B', 'E', 'B', 'D', 'C', 'A', 'B', 'B', 'C', 'B', 'C', 'B', 'A', 'D', 'B', 'C', 'B', 'D', 'B', 'D', 'A', 'D', 'B', 'A', 'D', 'E', 'E', 'B', 'A', 'A', 'E', 'D', 'E', 'C', 'B']
['C', 'E', 'E', 'C', 'D', 'B', 'D', 'A', 'D', 'E', 'B', 'C', 'C', 'E', 'B', 'A', 'B', 'E', 'C', 'E', 'A', 'C', 'B', 'D', 'A', 'B', 'D', 'B', 'D', 'D', 'A', 'B', 'D', 'D', 'E', 'C', 'A', 'D', 'C', 'C', 'E', 'A', 'C', 'B', 'A']
552.8


array([ 0, -4, -2,  1,  0,  2, -2,  0,  0, -3,  0,  2, -1, -1,  1,  0,  0,
       -3,  0, -3,  2, -1, -1,  0,  1,  1, -2,  2, -2,  0,  0,  2, -2, -3,
       -1,  2,  4, -2, -2, -2,  0,  3,  2,  1,  1])

array([ 0, -2,  0,  1, -3,  1, -2,  0, -1, -3,  1,  1,  1, -2,  1,  2,  0,
       -1,  0, -1,  0, -2, -1,  0,  1,  2, -3,  3, -2, -3,  2,  2, -2,  1,
        0, -1,  4, -3,  2, -2, -2,  0,  1,  3,  2])

array([ 2, -1, -3, -1,  0,  0,  0,  0,  0, -3,  1,  0,  2,  0,  1,  3,  1,
       -3, -2, -4,  0,  0, -1, -2,  2,  1, -3,  1, -3,  0,  4, -1, -3,  0,
       -1, -1,  2, -1,  0, -2, -1,  2, -2,  0,  2])

array([ 0,  0,  0,  0,  0,  0,  0,  1,  0, -4,  3, -1, -1, -1,  3,  3, -1,
       -1, -2, -4,  0, -2, -1, -3,  0, -1, -3, -1, -2, -1,  2, -1,  0, -3,
       -1,  0,  4, -1, -1,  0, -4,  1, -2, -1,  2])

...

array([ 0,  0,  0, -1,  0, -1, -2,  0, -1,  0,  0,  2,  0, -3,  0,  0,  0,
        0,  0, -2,  2,  0,  0,  0,  0,  0,  1,  2,  0,  0,  2,  3, -1,  0,
       -1,  0,  1,  0,  0,  0,  0,  0,  2,  0,  0])

array([ 0, -1, -1, -2, -1,  3, -2,  4,  1, -2,  2,  0,  1, -1,  0,  0,  2,
       -3,  0, -1,  3,  0,  1,  0,  1,  3, -2,  0, -2,  0,  1, -1, -2, -3,
       -1,  0,  0,  0, -1,  1, -2,  4, -1,  1,  2])

array([ 0,  0,  0,  2,  0,  0,  1,  0,  0, -1,  3, -2,  0, -2, -1,  0,  0,
       -4, -2, -4,  3,  0,  3,  1,  4,  2,  0,  0, -3, -3,  2,  3, -1,  1,
        0,  2,  3, -2,  0, -1, -4,  2,  1,  3,  3])

array([ 2, -3, -4,  1, -1,  3, -2,  0,  0, -4,  2,  2,  0, -3,  3,  1,  2,
       -2, -2, -2,  0,  2,  2, -3,  1,  2, -2,  0,  1, -3,  4, -1, -1, -3,
       -2,  1,  1, -3, -1, -2,  0,  2,  0,  0,  0])

array([ 1,  0, -4, -2,  0,  3,  0,  0, -1, -3,  3, -1, -1,  0,  2,  4,  1,
        0,  1, -3,  2,  2,  2,  1,  2,  2,  0,  1, -1, -3,  2,  2,  0, -2,
       -3,  1,  4, -3, -1,  1, -3,  4, -1,  1,  4])

In [20]:
for gab in gabarito.keys():
    data = student_data[student_data['CO_PROVA_MT'] == gab]
    print(f'Dados do gabarito {gab}:\n', data['NU_NOTA_MT'].describe())


Dados do gabarito 706866896147fc1fe97fb4351bc789a04b3e1ac8:
 count    2541.000000
mean      509.552105
std       107.952543
min         0.000000
25%       429.300000
50%       491.200000
75%       574.900000
max       933.000000
Name: NU_NOTA_MT, dtype: float64
Dados do gabarito 33afbd61183d36159e198672aea0957d3988f398:
 count    2511.000000
mean      498.922581
std       103.009797
min         0.000000
25%       420.100000
50%       480.200000
75%       559.300000
max       936.700000
Name: NU_NOTA_MT, dtype: float64
Dados do gabarito eaf8decfa5e293b30347d68e8c96bd308d1747e2:
 count    2701.000000
mean      498.436357
std       108.091976
min         0.000000
25%       416.000000
50%       473.600000
75%       562.200000
max       990.500000
Name: NU_NOTA_MT, dtype: float64
Dados do gabarito 7768972b2a16ffd60c566951859cb2742cbbea50:
 count    2255.000000
mean      502.740665
std       111.240206
min         0.000000
25%       416.500000
50%       478.600000
75%       568.350000
max   

In [21]:
distrib_gabarito = pd.DataFrame(gbn_one, columns=['rep'])
print(distrib_gabarito['rep'].value_counts() / 45)

NameError: name 'gbn_one' is not defined

In [22]:
max_nota_data = student_data[student_data['NU_NOTA_MT'] == 990.50]
display(max_nota_data[['NU_INSCRICAO','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TX_RESPOSTAS_MT', 'TX_GABARITO_MT']])

,NU_INSCRICAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_MT,TX_GABARITO_MT
11774,90ac84a3f0c7c6c7d44767ab8b63e0b5cc4b257c,714.8,641.3,624.9,990.5,CBAACCEABDDCDBDDCEDECADBECEBACBDABDBEBCDADCEE,CBAACCEABDDCDBDDCEAECADBECEBACBDABDBEBCDADCEE


In [3]:
size = 46

respostas_mt = student_data[['TX_RESPOSTAS_MT','CO_PROVA_MT']]
respostas_mt = respostas_mt.dropna()
columns=[f'Q{x}' for x in range(1, size)]
columns.extend(['CO_PROVA_MT'])

questions = pd.DataFrame(columns=columns)
for row_index, row in respostas_mt.iterrows():
    cor_prova = row['CO_PROVA_MT']
    serie = prepare_gb(list(row['TX_RESPOSTAS_MT']))    
    serie = np.append(serie, cor_prova)
    
    questions = questions.append(pd.Series(serie, index=columns), ignore_index=True) 


for col in columns:
    questions = questions.drop(questions[questions[f'{col}'] == 0].index)

In [4]:
questions = questions[(questions != "0").all(1)]

Q40 - acerto=15% - dificil
Q41 - acerto=24% - média
Q42 - acerto=24% - média
Q43 - acerto=11% - dificil
Q44 - acerto=29% - facil
Q45 - acerto=21% - média

In [105]:
for q in range(len(questions)):
    row = list(questions.iloc[q])
    x_total = row[:40]
    x_prob = [[x] for x in x_total]
    x_prob_a = [x for x in x_total]

    pd_sample = pd.DataFrame(x_prob, columns=['rep'])
    print(pd_sample['rep'].value_counts() / 45)
    #print(list(questions.iloc[q])[:45])


2    0.288889
4    0.222222
1    0.177778
3    0.133333
5    0.066667
Name: rep, dtype: float64
4    0.311111
3    0.200000
2    0.133333
5    0.133333
1    0.111111
Name: rep, dtype: float64
4    0.266667
3    0.200000
1    0.200000
5    0.111111
2    0.111111
Name: rep, dtype: float64
2    0.222222
3    0.200000
4    0.200000
5    0.177778
1    0.088889
Name: rep, dtype: float64
3    0.222222
1    0.200000
4    0.177778
2    0.155556
5    0.133333
Name: rep, dtype: float64
3    0.244444
1    0.222222
4    0.177778
2    0.155556
5    0.088889
Name: rep, dtype: float64
1    0.311111
4    0.177778
3    0.155556
2    0.133333
5    0.111111
Name: rep, dtype: float64
4    0.377778
1    0.177778
3    0.177778
2    0.133333
5    0.022222
Name: rep, dtype: float64
3    0.222222
1    0.200000
4    0.155556
2    0.155556
5    0.155556
Name: rep, dtype: float64
2    0.266667
4    0.222222
1    0.177778
3    0.133333
5    0.088889
Name: rep, dtype: float64
2    0.266667
3    0.200000
4    0.17777

Name: rep, dtype: float64
4    0.222222
2    0.200000
3    0.177778
5    0.155556
1    0.133333
Name: rep, dtype: float64
2    0.288889
3    0.200000
4    0.177778
5    0.111111
1    0.111111
Name: rep, dtype: float64
3    0.288889
1    0.200000
4    0.177778
2    0.155556
5    0.066667
Name: rep, dtype: float64
1    0.244444
3    0.200000
5    0.200000
2    0.177778
4    0.066667
Name: rep, dtype: float64
4    0.266667
3    0.222222
2    0.155556
5    0.133333
1    0.111111
Name: rep, dtype: float64
3    0.266667
1    0.266667
2    0.177778
4    0.111111
5    0.066667
Name: rep, dtype: float64
2    0.222222
3    0.200000
1    0.200000
5    0.133333
4    0.133333
Name: rep, dtype: float64
2    0.222222
4    0.200000
5    0.200000
3    0.200000
1    0.066667
Name: rep, dtype: float64
5    0.200000
3    0.200000
2    0.177778
1    0.177778
4    0.133333
Name: rep, dtype: float64
2    0.222222
3    0.222222
4    0.155556
1    0.155556
5    0.133333
Name: rep, dtype: float64
2    0.222222


3    0.311111
4    0.222222
5    0.200000
1    0.088889
2    0.066667
Name: rep, dtype: float64
1    0.333333
2    0.177778
5    0.155556
3    0.133333
4    0.088889
Name: rep, dtype: float64
3    0.533333
5    0.155556
2    0.111111
1    0.066667
4    0.022222
Name: rep, dtype: float64
4    0.200000
3    0.200000
5    0.177778
2    0.155556
1    0.155556
Name: rep, dtype: float64
3    0.222222
2    0.177778
5    0.177778
4    0.155556
1    0.155556
Name: rep, dtype: float64
2    0.311111
3    0.244444
1    0.177778
4    0.088889
5    0.066667
Name: rep, dtype: float64
4    0.222222
2    0.222222
1    0.177778
3    0.177778
5    0.088889
Name: rep, dtype: float64
2    0.244444
1    0.222222
3    0.177778
5    0.155556
4    0.088889
Name: rep, dtype: float64
1    0.266667
3    0.200000
2    0.177778
5    0.155556
4    0.088889
Name: rep, dtype: float64
3    0.288889
5    0.200000
4    0.133333
2    0.133333
1    0.133333
Name: rep, dtype: float64
3    0.222222
4    0.177778
1    0.17777

4    0.244444
2    0.222222
3    0.200000
5    0.111111
1    0.111111
Name: rep, dtype: float64
2    0.222222
3    0.222222
5    0.177778
1    0.155556
4    0.111111
Name: rep, dtype: float64
1    0.222222
2    0.200000
4    0.200000
5    0.133333
3    0.133333
Name: rep, dtype: float64
2    0.311111
3    0.177778
4    0.155556
1    0.133333
5    0.111111
Name: rep, dtype: float64
4    0.266667
5    0.200000
2    0.177778
3    0.177778
1    0.066667
Name: rep, dtype: float64
1    0.222222
3    0.222222
4    0.155556
5    0.155556
2    0.133333
Name: rep, dtype: float64
2    0.288889
1    0.200000
3    0.133333
5    0.133333
4    0.133333
Name: rep, dtype: float64
3    0.377778
4    0.288889
2    0.088889
5    0.088889
1    0.044444
Name: rep, dtype: float64
2    0.266667
4    0.155556
3    0.155556
5    0.155556
1    0.155556
Name: rep, dtype: float64
5    0.311111
1    0.177778
2    0.155556
3    0.133333
4    0.111111
Name: rep, dtype: float64
1    0.511111
4    0.133333
2    0.11111

4    0.244444
3    0.244444
1    0.222222
2    0.133333
5    0.044444
Name: rep, dtype: float64
3    0.244444
4    0.200000
2    0.200000
1    0.133333
5    0.111111
Name: rep, dtype: float64
4    0.311111
1    0.200000
2    0.177778
5    0.111111
3    0.088889
Name: rep, dtype: float64
1    0.222222
2    0.200000
3    0.177778
4    0.177778
5    0.111111
Name: rep, dtype: float64
3    0.266667
1    0.222222
2    0.177778
4    0.155556
5    0.066667
Name: rep, dtype: float64
4    0.200000
5    0.200000
3    0.200000
2    0.177778
1    0.111111
Name: rep, dtype: float64
2    0.288889
4    0.200000
3    0.177778
1    0.155556
5    0.066667
Name: rep, dtype: float64
2    0.244444
4    0.222222
3    0.155556
1    0.155556
5    0.111111
Name: rep, dtype: float64
3    0.311111
1    0.177778
2    0.155556
4    0.133333
5    0.111111
Name: rep, dtype: float64
3    0.266667
1    0.200000
4    0.177778
5    0.133333
2    0.111111
Name: rep, dtype: float64
1    0.333333
2    0.288889
5    0.11111

...

Name: rep, dtype: float64
4    0.222222
2    0.200000
3    0.200000
1    0.177778
5    0.088889
Name: rep, dtype: float64
1    0.244444
4    0.222222
5    0.155556
2    0.133333
3    0.133333
Name: rep, dtype: float64
2    0.333333
4    0.200000
3    0.155556
1    0.155556
5    0.044444
Name: rep, dtype: float64
1    0.355556
2    0.200000
3    0.177778
4    0.111111
5    0.044444
Name: rep, dtype: float64
1    0.288889
5    0.177778
4    0.177778
2    0.133333
3    0.111111
Name: rep, dtype: float64
3    0.311111
5    0.200000
2    0.177778
4    0.111111
1    0.088889
Name: rep, dtype: float64
1    0.288889
5    0.244444
3    0.200000
2    0.088889
4    0.066667
Name: rep, dtype: float64
3    0.266667
2    0.222222
1    0.177778
4    0.155556
5    0.066667
Name: rep, dtype: float64
3    0.244444
2    0.222222
4    0.177778
1    0.133333
5    0.111111
Name: rep, dtype: float64
3    0.244444
4    0.200000
1    0.200000
2    0.155556
5    0.088889
Name: rep, dtype: float64
3    0.266667


Name: rep, dtype: float64
1    0.266667
2    0.244444
4    0.133333
3    0.133333
5    0.111111
Name: rep, dtype: float64
3    0.266667
1    0.200000
2    0.177778
5    0.133333
4    0.111111
Name: rep, dtype: float64
4    0.288889
2    0.222222
3    0.155556
1    0.133333
5    0.088889
Name: rep, dtype: float64
2    0.311111
3    0.155556
5    0.155556
4    0.155556
1    0.111111
Name: rep, dtype: float64
5    0.266667
4    0.222222
1    0.200000
2    0.111111
3    0.088889
Name: rep, dtype: float64
3    0.288889
4    0.244444
2    0.133333
1    0.133333
5    0.088889
Name: rep, dtype: float64
4    0.266667
2    0.200000
1    0.155556
3    0.155556
5    0.111111
Name: rep, dtype: float64
2    0.266667
4    0.266667
1    0.133333
3    0.111111
5    0.111111
Name: rep, dtype: float64
3    0.266667
4    0.177778
2    0.177778
1    0.155556
5    0.111111
Name: rep, dtype: float64
5    0.244444
3    0.222222
4    0.155556
2    0.155556
1    0.111111
Name: rep, dtype: float64
4    0.266667


2    0.200000
1    0.200000
4    0.177778
3    0.177778
5    0.133333
Name: rep, dtype: float64
4    0.266667
5    0.222222
2    0.200000
3    0.111111
1    0.088889
Name: rep, dtype: float64
1    0.333333
4    0.155556
5    0.133333
2    0.133333
3    0.133333
Name: rep, dtype: float64
4    0.288889
1    0.200000
3    0.155556
5    0.133333
2    0.111111
Name: rep, dtype: float64
1    0.222222
4    0.177778
2    0.177778
5    0.177778
3    0.133333
Name: rep, dtype: float64
1    0.266667
2    0.200000
4    0.155556
3    0.155556
5    0.111111
Name: rep, dtype: float64
3    0.311111
4    0.244444
2    0.133333
1    0.133333
5    0.066667
Name: rep, dtype: float64
4    0.222222
5    0.222222
2    0.200000
3    0.177778
1    0.066667
Name: rep, dtype: float64
3    0.311111
4    0.200000
2    0.177778
5    0.111111
1    0.088889
Name: rep, dtype: float64
3    0.244444
5    0.244444
1    0.200000
4    0.111111
2    0.088889
Name: rep, dtype: float64
5    0.244444
3    0.222222
1    0.20000

Name: rep, dtype: float64
3    0.288889
2    0.222222
5    0.133333
4    0.133333
1    0.111111
Name: rep, dtype: float64
2    0.266667
1    0.177778
3    0.177778
5    0.133333
4    0.133333
Name: rep, dtype: float64
2    0.222222
3    0.222222
5    0.177778
1    0.177778
4    0.088889
Name: rep, dtype: float64
1    0.266667
2    0.200000
5    0.177778
3    0.133333
4    0.111111
Name: rep, dtype: float64
2    0.244444
3    0.177778
4    0.155556
5    0.155556
1    0.155556
Name: rep, dtype: float64
3    0.244444
1    0.177778
4    0.155556
2    0.155556
5    0.155556
Name: rep, dtype: float64
4    0.244444
3    0.200000
2    0.177778
1    0.155556
5    0.111111
Name: rep, dtype: float64
3    0.222222
4    0.222222
2    0.200000
1    0.155556
5    0.088889
Name: rep, dtype: float64
2    0.244444
5    0.200000
1    0.155556
4    0.155556
3    0.133333
Name: rep, dtype: float64
4    0.222222
3    0.222222
5    0.177778
1    0.155556
2    0.111111
Name: rep, dtype: float64
2    0.333333


KeyboardInterrupt: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

for gab in gabarito.keys():
    gab_questions = questions[questions['CO_PROVA_MT'] == gab]
    print(f'PROVA {gab}')
    for q in range(0, 6):
        counts = gab_questions[f'Q4{q}'].value_counts()
        percent_q = (counts / len(gab_questions))
        #display(list(percent_q))

    X = gab_questions.loc[:,:'Q40']
    y = gab_questions['Q41']

    
    decision_tree_classifier = DecisionTreeClassifier()
    min_samples_splits = np.linspace(0.1, 1.0, 10, endpoint=True)
    
    parameter_grid = {'max_depth': [x for x in range(1,6)],
                      'max_features': [x for x in range(1,6)],
                      'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True)}

    cross_validation = StratifiedShuffleSplit(n_splits=10, test_size=.3)

    grid_search = GridSearchCV(decision_tree_classifier, param_grid = parameter_grid,
                              cv = cross_validation)
    grid_search.fit(X, y)

    print ("Best Score: {}".format(grid_search.best_score_))
    print ("Best params: {}".format(grid_search.best_params_))
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    #clf = DecisionTreeClassifier()
    #clf = clf.fit(X_train, y_train)    
    #y_pred = clf.predict(X_test)
    
    #print(accuracy_score(y_test, y_pred))
    
    

In [21]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB 
from sklearn.metrics import accuracy_score

for gab in gabarito.keys():
    gab_questions = questions[questions['CO_PROVA_MT'] == gab]
    print(f'PROVA {gab}')
    for q in range(0, 6):
        counts = gab_questions[f'Q4{q}'].value_counts()
        percent_q = (counts / len(gab_questions))
        #display(list(percent_q))

    
    X = gab_questions.loc[:,:'Q40']
    X = X.assign(NU_NOTA_MT=gab_questions['NU_NOTA_MT'])
    y = gab_questions['Q41']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    clf = GaussianNB ()
    clf = clf.fit(X_train, y_train)    

    y_pred = clf.predict(X_test)
    
    print(accuracy_score(y_test, y_pred))
    print(clf.class_prior_)
    
    

PROVA 706866896147fc1fe97fb4351bc789a04b3e1ac8
0.2982917214191853
[0.22783172 0.2776699  0.17475728 0.17022654 0.14951456]
PROVA 33afbd61183d36159e198672aea0957d3988f398
0.26897470039946736
[0.21799081 0.23571898 0.27314511 0.17531188 0.09783322]
PROVA eaf8decfa5e293b30347d68e8c96bd308d1747e2
0.28553770086526575
[0.35079172 0.18514007 0.15408039 0.13215591 0.17783191]


In [73]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

for gab in gabarito.keys():
    gab_questions = questions[questions['CO_PROVA_MT'] == gab]
    print(f'PROVA {gab}')
    for q in range(0, 6):
        counts = gab_questions[f'Q4{q}'].value_counts()
        percent_q = (counts / len(gab_questions))
        #display(list(percent_q))

    
    X = gab_questions.loc[:,:'Q40']
    X = X.assign(NU_NOTA_MT=gab_questions['NU_NOTA_MT'])
    y = gab_questions['Q41']
    
    cross_validation = StratifiedShuffleSplit(n_splits=10, test_size=.3)

    parameters = [{
        'n_neighbors': [1, 3, 5, 10, 50, 100],
        'weights': ['uniform', 'distance']
    }]
    n_folds = 10

    clf = GridSearchCV(
        KNeighborsClassifier(), parameters, cv=cross_validation,
        scoring="accuracy"
    )
    clf.fit(X, y)
    
    print ("Best Score: {}".format(clf.best_score_))
    print ("Best params: {}".format(clf.best_params_))

PROVA 706866896147fc1fe97fb4351bc789a04b3e1ac8


KeyError: 'NU_NOTA_MT'

In [76]:
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import SGDClassifier
    
for gab in gabarito.keys():
    gab_questions = questions[questions['CO_PROVA_MT'] == gab]
    print(f'PROVA {gab}')
    for q in range(0, 6):
        counts = gab_questions[f'Q4{q}'].value_counts()
        percent_q = (counts / len(gab_questions))
        #display(list(percent_q))

    
    X = gab_questions.loc[:,:'Q40']
    #X = X.assign(NU_NOTA_MT=gab_questions['NU_NOTA_MT'])
    y = gab_questions['Q41']
    
    cross_validation = StratifiedShuffleSplit(n_splits=10, test_size=.3)

    parameters = [{
        'loss': ('log', 'hinge'),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'alpha': [0.001, 0.0001, 0.00001, 0.000001]
    }]
    
    n_folds = 10

    clf = GridSearchCV(
        SGDClassifier(), parameters, cv=cross_validation,
        scoring="accuracy"
    )
    clf.fit(X, y)
    
    print ("Best Score: {}".format(clf.best_score_))
    print ("Best params: {}".format(clf.best_params_))

PROVA 706866896147fc1fe97fb4351bc789a04b3e1ac8
Best Score: 0.24692005242463957
Best params: {'alpha': 1e-06, 'loss': 'hinge', 'penalty': 'l1'}
PROVA 33afbd61183d36159e198672aea0957d3988f398
Best Score: 0.25013262599469493
Best params: {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l1'}
PROVA eaf8decfa5e293b30347d68e8c96bd308d1747e2
Best Score: 0.26942046855733665
Best params: {'alpha': 1e-06, 'loss': 'hinge', 'penalty': 'elasticnet'}


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.metrics import accuracy_score, f1_score
from sklearn.neural_network import MLPClassifier
    
for gab in gabarito.keys():
    gab_questions = questions[questions['CO_PROVA_MT'] == gab]

    gab_questions = gab_questions[:300]
    
    print(f'PROVA {gab}')
    for q in range(0, 6):
        counts = gab_questions[f'Q4{q}'].value_counts()
        percent_q = (counts / len(gab_questions))
        #display(list(percent_q))

    
    X = gab_questions.loc[:,:'Q40']
    #X = X.assign(NU_NOTA_MT=gab_questions['NU_NOTA_MT'])
    y = gab_questions['Q41']
    
    cross_validation = StratifiedShuffleSplit(n_splits=10, test_size=.3)

    parameters = [{        
        'activation' : ['identity', 'logistic', 'tanh', 'relu'],
        'hidden_layer_sizes': (2,5,10,15), 
        'solver' : ['lbfgs', 'sgd', 'adam'],
        'random_state': [1, 42], 
        'max_iter': [1, 10, 100, 500], 
        #'alpha': [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]
        'alpha': [10000, 1000, 100, 10, 1]
    }]
    
    clf = GridSearchCV(
        MLPClassifier(), parameters, cv=cross_validation,
        scoring="accuracy"
    )
    clf.fit(X, y)
    
    print ("Best Score: {}".format(clf.best_score_))
    print ("Best params: {}".format(clf.best_params_))

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q37,Q38,Q39,Q40,Q41,Q42,Q43,Q44,Q45,CO_PROVA_MT
0,3,1,3,4,4,4,2,1,4,2,...,5,2,1,1,5,4,5,3,2,706866896147fc1fe97fb4351bc789a04b3e1ac8
4,3,3,5,4,1,3,2,1,3,2,...,5,1,5,1,3,1,4,5,3,706866896147fc1fe97fb4351bc789a04b3e1ac8
5,5,4,2,2,4,2,4,1,4,2,...,3,3,3,1,4,3,1,2,3,706866896147fc1fe97fb4351bc789a04b3e1ac8
6,3,5,5,3,4,2,4,2,4,1,...,5,3,2,3,1,2,1,1,3,706866896147fc1fe97fb4351bc789a04b3e1ac8
7,3,1,4,4,4,2,4,1,4,1,...,3,3,4,4,2,4,4,4,4,706866896147fc1fe97fb4351bc789a04b3e1ac8
13,3,5,5,2,4,1,2,1,3,5,...,2,4,3,3,5,1,5,2,1,706866896147fc1fe97fb4351bc789a04b3e1ac8
15,3,4,4,1,3,5,2,5,5,3,...,1,4,2,4,3,5,2,3,3,706866896147fc1fe97fb4351bc789a04b3e1ac8
17,3,5,5,5,4,2,5,1,4,4,...,4,2,3,2,1,3,4,5,4,706866896147fc1fe97fb4351bc789a04b3e1ac8
18,5,2,1,4,3,5,2,1,4,1,...,2,1,2,1,5,3,3,2,1,706866896147fc1fe97fb4351bc789a04b3e1ac8
20,4,5,1,1,4,5,4,1,3,2,...,5,1,2,4,2,5,2,3,5,706866896147fc1fe97fb4351bc789a04b3e1ac8


PROVA 706866896147fc1fe97fb4351bc789a04b3e1ac8
Best Score: 0.27111111111111114
Best params: {'activation': 'relu', 'alpha': 10000, 'hidden_layer_sizes': 10, 'max_iter': 1, 'random_state': 1, 'solver': 'sgd'}


,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q37,Q38,Q39,Q40,Q41,Q42,Q43,Q44,Q45,CO_PROVA_MT
1,4,2,4,5,3,1,2,3,2,4,...,3,4,1,2,1,4,2,3,1,33afbd61183d36159e198672aea0957d3988f398
3,4,2,4,5,5,1,3,4,4,5,...,2,2,5,3,2,2,1,2,2,33afbd61183d36159e198672aea0957d3988f398
16,3,2,3,5,4,3,2,2,5,4,...,2,1,4,2,4,5,5,5,2,33afbd61183d36159e198672aea0957d3988f398
19,3,3,3,2,2,3,2,3,4,4,...,1,1,4,3,4,4,1,2,2,33afbd61183d36159e198672aea0957d3988f398
23,4,1,3,1,3,1,3,5,4,3,...,1,3,3,3,4,3,1,3,2,33afbd61183d36159e198672aea0957d3988f398
25,4,1,5,2,3,2,1,4,2,3,...,2,4,4,1,5,4,1,3,2,33afbd61183d36159e198672aea0957d3988f398
38,1,1,3,4,1,3,4,4,2,3,...,2,3,5,3,1,1,1,2,3,33afbd61183d36159e198672aea0957d3988f398
39,3,5,4,5,3,2,2,4,3,4,...,1,1,5,4,2,4,2,5,3,33afbd61183d36159e198672aea0957d3988f398
42,3,5,2,4,3,1,2,1,2,4,...,3,4,5,4,2,4,1,4,3,33afbd61183d36159e198672aea0957d3988f398
45,2,4,1,3,1,4,4,2,1,2,...,3,4,2,3,5,3,3,5,2,33afbd61183d36159e198672aea0957d3988f398


PROVA 33afbd61183d36159e198672aea0957d3988f398


# preparação dos dados reais

In [106]:
real_data = pd.read_csv("test3.csv")

In [107]:
size = 41

respostas_mt = real_data[['TX_RESPOSTAS_MT','CO_PROVA_MT','NU_INSCRICAO']]
respostas_mt = respostas_mt.dropna()
columns=[f'Q{x}' for x in range(1, size)]
columns.extend(['CO_PROVA_MT','NU_INSCRICAO'])

real_questions = pd.DataFrame(columns=columns)
for row_index, row in respostas_mt.iterrows():
    cor_prova = row['CO_PROVA_MT']
    inscricao = row['NU_INSCRICAO']
    
    serie = prepare_gb(list(row['TX_RESPOSTAS_MT']))    
    serie = np.append(serie, cor_prova)
    serie = np.append(serie, inscricao)

    real_questions = real_questions.append(pd.Series(serie, index=columns), ignore_index=True) 


for col in columns:
    real_questions = real_questions.drop(real_questions[real_questions[f'{col}'] == 0].index)

In [108]:
display(questions['CO_PROVA_MT'].unique())

array(['706866896147fc1fe97fb4351bc789a04b3e1ac8',
       '33afbd61183d36159e198672aea0957d3988f398',
       'eaf8decfa5e293b30347d68e8c96bd308d1747e2',
       '7768972b2a16ffd60c566951859cb2742cbbea50',
       '533c7c38726cb52f1075e2699c3e7d6b4b95a2e5',
       'c5d07ed2a1535f074eb558a440b069b5eefa6246',
       'ab155449f2d11d09b533b92c9547aaf26a264a34',
       'e0406d1d2dd9b6f07e2db00602f9dc663450ea51',
       '284fac346c24f267c62cf9070dd8bb1ee23e213c'], dtype=object)

In [120]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

master_response = pd.DataFrame(columns=['NU_INSCRICAO','Q41','Q42','Q43','Q44','Q45'])

for gab in gabarito.keys():
    gab_questions = questions[questions['CO_PROVA_MT'] == gab]
    print(f'PROVA {gab}')
    for q in range(0, 6):
        counts = gab_questions[f'Q4{q}'].value_counts()
        percent_q = (counts / len(gab_questions))
        #display(list(percent_q))

    x_predict = real_questions[real_questions['CO_PROVA_MT'] == gab]
    
    gab_response = pd.DataFrame(columns=['NU_INSCRICAO','Q41','Q42','Q43','Q44','Q45'])
    gab_response = gab_response.assign(NU_INSCRICAO=x_predict['NU_INSCRICAO'])
    
    
    for i in range(1,6):
        print(f"     Treinando questão 4{i}....")
        
        X = gab_questions.loc[:,:'Q40']
        y = gab_questions[f'Q4{i}']


        decision_tree_classifier = DecisionTreeClassifier()

        parameter_grid = {'max_depth': [x for x in range(1,6)],
                          'max_features': [x for x in range(1,6)],
                          'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True)}

        cross_validation = StratifiedShuffleSplit(n_splits=10, test_size=.3)

        grid_search = GridSearchCV(decision_tree_classifier, param_grid = parameter_grid,
                                  cv = cross_validation)
        grid_search.fit(X, y)

        print ("Best Score: {}".format(grid_search.best_score_))
        print ("Best params: {}".format(grid_search.best_params_))   
        
        
        clf = grid_search.best_estimator_
        
        #x_predict.loc[:,:'Q40']
        y_pred = clf.predict(x_predict.loc[:,:'Q40'])
        gab_response[f'Q4{i}'] = y_pred
        

    master_response = master_response.append(gab_response)

#display(master_response)

PROVA 706866896147fc1fe97fb4351bc789a04b3e1ac8
     Treinando questão 41....
Best Score: 0.28627167630057804
Best params: {'max_depth': 2, 'max_features': 2, 'min_samples_split': 0.30000000000000004}
     Treinando questão 42....
Best Score: 0.38771676300578034
Best params: {'max_depth': 2, 'max_features': 3, 'min_samples_split': 0.4}
     Treinando questão 43....
Best Score: 0.309393063583815
Best params: {'max_depth': 2, 'max_features': 5, 'min_samples_split': 0.2}
     Treinando questão 44....
Best Score: 0.2526011560693642
Best params: {'max_depth': 1, 'max_features': 1, 'min_samples_split': 0.8}
     Treinando questão 45....
Best Score: 0.25289017341040465
Best params: {'max_depth': 5, 'max_features': 5, 'min_samples_split': 0.30000000000000004}
PROVA 33afbd61183d36159e198672aea0957d3988f398
     Treinando questão 41....
Best Score: 0.29267935578330895
Best params: {'max_depth': 3, 'max_features': 5, 'min_samples_split': 0.6}
     Treinando questão 42....
Best Score: 0.28023426061

,NU_INSCRICAO,Q41,Q42,Q43,Q44,Q45


In [112]:
real_gab = ['706866896147fc1fe97fb4351bc789a04b3e1ac8',
       '33afbd61183d36159e198672aea0957d3988f398',
       'eaf8decfa5e293b30347d68e8c96bd308d1747e2',
       '7768972b2a16ffd60c566951859cb2742cbbea50',
       '533c7c38726cb52f1075e2699c3e7d6b4b95a2e5',
       'c5d07ed2a1535f074eb558a440b069b5eefa6246',
       'ab155449f2d11d09b533b92c9547aaf26a264a34',
       'e0406d1d2dd9b6f07e2db00602f9dc663450ea51',
       '284fac346c24f267c62cf9070dd8bb1ee23e213c']

gab_rest = real_gab - gabarito.keys() 

In [113]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

gab_questions = questions

x_predict = real_questions[real_questions['CO_PROVA_MT'].isin(gab_rest)]

    
gab_response = pd.DataFrame(columns=['NU_INSCRICAO','Q41','Q42','Q43','Q44','Q45'])
gab_response = gab_response.assign(NU_INSCRICAO=x_predict['NU_INSCRICAO'])
        
for i in range(1,6):
    print(f"     Treinando questão 4{i}....")

    X = gab_questions.loc[:,:'Q40']
    y = gab_questions[f'Q4{i}']


    decision_tree_classifier = DecisionTreeClassifier()

    parameter_grid = {'max_depth': [x for x in range(1,6)],
                      'max_features': [x for x in range(1,6)]}

    cross_validation = StratifiedShuffleSplit(n_splits=10, test_size=.3)

    grid_search = GridSearchCV(decision_tree_classifier, param_grid = parameter_grid,
                              cv = cross_validation)
    grid_search.fit(X, y)

    print ("Best Score: {}".format(grid_search.best_score_))
    print ("Best params: {}".format(grid_search.best_params_))   
        
    clf = grid_search.best_estimator_

        #x_predict.loc[:,:'Q40']

    y_pred = clf.predict(x_predict.loc[:,:'Q40'])
    gab_response[f'Q4{i}'] = y_pred
    
master_response = master_response.append(gab_response)

#display(master_response)

     Treinando questão 41....
Best Score: 0.27135714285714285
Best params: {'max_depth': 3, 'max_features': 3}


ValueError: Found array with 0 sample(s) (shape=(0, 40)) while a minimum of 1 is required.

In [121]:
len(master_response)
#len(x_predict[x_predict['CO_PROVA_MT'].isin(gabarito.keys())])


3265

In [122]:
master_response = master_response.loc[:,:'Q45'].replace(['1','2','3','4','5'],['A','B','C','D','E'])
concat_data = []
master_response['TX_RESPOSTAS_MT'] = master_response["Q41"] + master_response["Q42"] + master_response["Q43"] + master_response["Q44"] + master_response["Q45"].map(str)

#concat_data = pd.DataFrame(concat_data, columns=['concat_data'])
#master_response['CONCAT_DATA'] = concat_data

In [123]:
print(master_response)

                                  NU_INSCRICAO Q41 Q42 Q43 Q44 Q45  \
2     b758c6c0905947e3f29b6aaa7a8343a0dad089bd   B   C   C   B   A   
8     2f2ea1ff47d418511ff8d32f6c6fe23a74bfcdb7   A   C   C   B   D   
15    a3b6183756b82df4ee88a0af7a67075c923c0542   B   C   C   B   D   
20    0d08de733b33f01b80dc9780172d4324f1df6ba6   B   C   C   B   A   
22    6d7b28e7db8a52fa06961d92b18dba2ae70f30ef   B   C   C   B   B   
25    66b615b9811d33935699f36854d1c5d78bec849d   A   C   C   B   A   
28    8134fbcc4fdbcca3b4a0414f867187a10e8c5f50   B   C   C   B   A   
29    3cd42d1780efd166b2f91094d8e09f8a03a40256   B   C   C   B   D   
34    068dcce5dbca69995c966eb7529949736b7ebc65   B   C   B   B   A   
37    1cb4498d8687ee7374f27550c1588c52b6458d8f   B   C   C   B   D   
39    83a7e51c8afa44f5a11200dbc247df257680be84   B   C   C   B   B   
42    e424dde6b04ff5fbbd2ece0bdb70c90848e91717   B   C   C   B   D   
47    6c2cab8d0107e9161ca5fd6e61ac831a4a624d90   B   C   C   B   A   
49    2f8319e426bdc4

In [125]:
send_data = master_response[['NU_INSCRICAO']]

answer = json.loads(master_response[['NU_INSCRICAO', 'TX_RESPOSTAS_MT']].to_json(orient="records"))
#np.average(student_data["NU_NOTA_MT"])
data_post = dict(
    token="40d8e606ca7a56e00ebfffd548fcd44f5c648bf7",
    email="paulorodriguesxv@gmail.com",
    answer=answer)

print(data_post)


{'token': '40d8e606ca7a56e00ebfffd548fcd44f5c648bf7', 'email': 'paulorodriguesxv@gmail.com', 'answer': [{'NU_INSCRICAO': 'b758c6c0905947e3f29b6aaa7a8343a0dad089bd', 'TX_RESPOSTAS_MT': 'BCCBA'}, {'NU_INSCRICAO': '2f2ea1ff47d418511ff8d32f6c6fe23a74bfcdb7', 'TX_RESPOSTAS_MT': 'ACCBD'}, {'NU_INSCRICAO': 'a3b6183756b82df4ee88a0af7a67075c923c0542', 'TX_RESPOSTAS_MT': 'BCCBD'}, {'NU_INSCRICAO': '0d08de733b33f01b80dc9780172d4324f1df6ba6', 'TX_RESPOSTAS_MT': 'BCCBA'}, {'NU_INSCRICAO': '6d7b28e7db8a52fa06961d92b18dba2ae70f30ef', 'TX_RESPOSTAS_MT': 'BCCBB'}, {'NU_INSCRICAO': '66b615b9811d33935699f36854d1c5d78bec849d', 'TX_RESPOSTAS_MT': 'ACCBA'}, {'NU_INSCRICAO': '8134fbcc4fdbcca3b4a0414f867187a10e8c5f50', 'TX_RESPOSTAS_MT': 'BCCBA'}, {'NU_INSCRICAO': '3cd42d1780efd166b2f91094d8e09f8a03a40256', 'TX_RESPOSTAS_MT': 'BCCBD'}, {'NU_INSCRICAO': '068dcce5dbca69995c966eb7529949736b7ebc65', 'TX_RESPOSTAS_MT': 'BCBBA'}, {'NU_INSCRICAO': '1cb4498d8687ee7374f27550c1588c52b6458d8f', 'TX_RESPOSTAS_MT': 'BCCBD

In [126]:
url = "https://api.codenation.com.br/v1/user/acceleration/data-science/challenge/enem-3/submit"

import requests

r = requests.post(url, json=data_post)

print(r.json())

{'score': 31.62278015921617}


In [163]:

for gab in gabarito.keys():
    gab_questions = questions[questions['CO_PROVA_MT'] == gab]
    print(f'PROVA {gab}')
    
    for i in range(1, 6):
        points = gab_questions[f'Q{i}'].value_counts() / len(gab_questions['Q1'])
        print(f'  Questão Q{i}: \n', points)

PROVA 706866896147fc1fe97fb4351bc789a04b3e1ac8
  Questão Q1: 
 3    0.720295
5    0.088465
2    0.085863
4    0.054206
1    0.051171
Name: Q1, dtype: float64
  Questão Q2: 
 1    0.336513
3    0.179965
2    0.176496
5    0.155247
4    0.151778
Name: Q2, dtype: float64
  Questão Q3: 
 5    0.401561
3    0.192541
2    0.166522
4    0.135299
1    0.104076
Name: Q3, dtype: float64
  Questão Q4: 
 4    0.395490
3    0.215091
2    0.166522
5    0.141804
1    0.081093
Name: Q4, dtype: float64
  Questão Q5: 
 4    0.553339
3    0.161752
5    0.108846
2    0.097138
1    0.078925
Name: Q5, dtype: float64
PROVA 33afbd61183d36159e198672aea0957d3988f398
  Questão Q1: 
 4    0.280563
1    0.269569
2    0.211961
3    0.134565
5    0.103342
Name: Q1, dtype: float64
  Questão Q2: 
 1    0.423923
5    0.260334
2    0.126649
3    0.102902
4    0.086192
Name: Q2, dtype: float64
  Questão Q3: 
 4    0.462181
3    0.189534
5    0.126649
2    0.125770
1    0.095866
Name: Q3, dtype: float64
  Questão Q4: 
 3 

In [191]:
dict_asw = {
    '706866896147fc1fe97fb4351bc789a04b3e1ac8':'CACDD',    
    '33afbd61183d36159e198672aea0957d3988f398':'DADCC',
    'eaf8decfa5e293b30347d68e8c96bd308d1747e2':'CBACC',
    '7768972b2a16ffd60c566951859cb2742cbbea50': 'CCCCC',
    'ab155449f2d11d09b533b92c9547aaf26a264a34': 'CCCCC',
    'c5d07ed2a1535f074eb558a440b069b5eefa6246': 'CCCCC',
    'e0406d1d2dd9b6f07e2db00602f9dc663450ea51': 'CCCCC',
    '533c7c38726cb52f1075e2699c3e7d6b4b95a2e5': 'CCCCC',}

In [192]:
master_response2 = []
for i, row in real_questions.iterrows():
    master_response2.append(dict(NU_INSCRICAO=row['NU_INSCRICAO'], TX_RESPOSTAS_MT=dict_asw[row['CO_PROVA_MT']] ))

In [197]:
#np.average(student_data["NU_NOTA_MT"])
data_post = dict(
    token="40d8e606ca7a56e00ebfffd548fcd44f5c648bf7",
    email="paulorodriguesxv@gmail.com",
    answer=master_response2)

print(data_post)


{'token': '40d8e606ca7a56e00ebfffd548fcd44f5c648bf7', 'email': 'paulorodriguesxv@gmail.com', 'answer': [{'NU_INSCRICAO': '05aafa04d90688082d947b1fa5f2928db9f058dd', 'TX_RESPOSTAS_MT': 'DADCC'}, {'NU_INSCRICAO': '4df5a16760633b75843b2461065b86aaa04b36a7', 'TX_RESPOSTAS_MT': 'CBACC'}, {'NU_INSCRICAO': 'b758c6c0905947e3f29b6aaa7a8343a0dad089bd', 'TX_RESPOSTAS_MT': 'CACDD'}, {'NU_INSCRICAO': 'de1c0464b060305860569640e03c406970e46679', 'TX_RESPOSTAS_MT': 'DADCC'}, {'NU_INSCRICAO': 'ddc451a56f34f762437a754e4c123ed47e87e1c9', 'TX_RESPOSTAS_MT': 'CCCCC'}, {'NU_INSCRICAO': 'dc5468021e84ff91ff8a2fedcdedeb16ce9dc23f', 'TX_RESPOSTAS_MT': 'DADCC'}, {'NU_INSCRICAO': '3df154f83284bbf0baafd395d3218476d9261cde', 'TX_RESPOSTAS_MT': 'DADCC'}, {'NU_INSCRICAO': '3b9f0e1348e2af7abc18f5292fbc29b7d3226f4d', 'TX_RESPOSTAS_MT': 'CCCCC'}, {'NU_INSCRICAO': '2f2ea1ff47d418511ff8d32f6c6fe23a74bfcdb7', 'TX_RESPOSTAS_MT': 'CACDD'}, {'NU_INSCRICAO': 'fbdc0166517805efa434bdf61695d6e5df15af1a', 'TX_RESPOSTAS_MT': 'CCCCC

In [198]:
url = "https://api.codenation.com.br/v1/user/acceleration/data-science/challenge/enem-3/submit"

import requests

r = requests.post(url, json=data_post)

print(r.json())

{'score': 18.744641763625232}
